In [34]:
!pip install geomdl

In [35]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

import numpy as np
import mshr
import fenics as fe
from geomdl import NURBS
from geomdl import utilities
from geomdl.visualization import VisMPL as vis

In [36]:
def get_linspace(start, end, num_points):
  return np.linspace(start, end, num_points)

In [37]:
def to_meters(size):
  return size * 10**(-6)

In [38]:
def generate_curve(control_points):
  curve = NURBS.Curve()
  curve.degree = 3
  curve.ctrlpts = control_points
  curve.knotvector = [0, 0, 0, 0] + [i for i in range(1, len(control_points)-3)] + [len(control_points)-3] * 4
  # curve.vis = vis.VisCurve2D()
  # curve.render()
  evaluated_points = curve.evalpts
  x_values = [pt[0] for pt in evaluated_points]
  y_values = [pt[1] for pt in evaluated_points]
  return x_values, y_values

In [39]:
def create_healthy_domain(length, height, undulation_amplitude, undulation_frequency, oscilations):

  k, f = undulation_amplitude, undulation_frequency

  x_vals = get_linspace(0, length, 100)
  y_vals_top = height + np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals) / length) for i in range(1, oscilations+1)], axis=0)
  y_vals_bottom = np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals + to_meters(0.1) * (-1)**i) / length) for i in range(1, oscilations)], axis=0)

  top_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals[::-1], y_vals_top[::-1])]
  bottom_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals, y_vals_bottom)]
  vessel_boundary_points = top_boundary_points + bottom_boundary_points

  vessel_domain = mshr.Polygon(vessel_boundary_points)

  return vessel_domain

In [40]:
def create_isthmus_domain(length, height, isthmus_height, isthmus_width, isthmus_position, undulation_amplitude, undulation_frequency):

  k, f = undulation_amplitude, undulation_frequency
  f *= 50000

  x_vals = get_linspace(0, length, 100)
  y_vals_top = height - (isthmus_height * np.exp(-((x_vals+to_meters(0.2)) - isthmus_position)**2 / (2 * isthmus_width**2)) + k * np.sin(2 * np.pi * f * x_vals+to_meters(0.3)))
  y_vals_bottom = isthmus_height * np.exp(-(x_vals - isthmus_position)**2 / (2 * isthmus_width**2)) + k * (np.sin(2 * np.pi * f * x_vals) + 0.5*np.sin(2 * np.pi * 2*f * x_vals))

  top_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals[::-1], y_vals_top[::-1])]
  bottom_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals, y_vals_bottom)]
  vessel_boundary_points = top_boundary_points + bottom_boundary_points

  vessel_domain = mshr.Polygon(vessel_boundary_points)

  return vessel_domain

In [41]:
def create_branch_domain(length, height, branch_width, branch_angle, branch_position_x, undulation_amplitude, undulation_frequency, oscilations):

  main_domain = create_healthy_domain(length, height, undulation_amplitude, undulation_frequency, oscilations)
  k, f = undulation_amplitude, undulation_frequency

  x_vals = get_linspace(branch_position_x, length, 100)

  y_vals_bottom_main = np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals + to_meters(0.1) * (-1)**i) / length) for i in range(1, oscilations)], axis=0)
  y_max = y_vals_bottom_main.max()
  y_vals_top_main = np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals) / length) for i in range(1, oscilations+1)], axis=0)
  y_min = np.abs(y_vals_top_main.max()) + np.abs(y_vals_top_main.min())

  y_vals_top = branch_width + np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals-branch_position_x) / length) for i in range(1, oscilations+1)], axis=0)
  y_vals_top = np.tan(branch_angle) * (x_vals-branch_position_x) + y_vals_top
  y_vals_top = np.array(y_vals_top) - y_min
  y_vals_bottom = np.sum([i * k * np.sin(2 * np.pi * f/i * ((x_vals-branch_position_x) + to_meters(0.1) * (-1)**i) / length) for i in range(1, oscilations)], axis=0)
  y_vals_bottom = np.tan(branch_angle) * (x_vals-branch_position_x) + y_vals_bottom + to_meters(0.1)
  y_vals_bottom = y_max + np.array(y_vals_bottom)

  top_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals[::-1], y_vals_top[::-1])]
  bottom_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals, y_vals_bottom)]
  vessel_boundary_points = top_boundary_points + bottom_boundary_points

  branch_domain = mshr.Polygon(vessel_boundary_points)
  vessel_domain = main_domain + branch_domain

  return vessel_domain

In [42]:
def create_aneurysm_domain(length, height, aneurysm_width, aneurysm_height, aneurysm_strecth, aneurysm_intake, aneurysm_roundness, aneurysm_center, undulation_amplitude, undulation_frequency, oscilations):

  main_domain = create_healthy_domain(length, height, undulation_amplitude, undulation_frequency, oscilations)
  k, f = undulation_amplitude, undulation_frequency

  control_points = [
      [-aneurysm_width/4 - aneurysm_intake, 0, 0],
       [-aneurysm_width/12 - aneurysm_intake/2, 0, 0],
        [-aneurysm_width/10 - aneurysm_roundness/3, aneurysm_strecth, 0],
         [-aneurysm_width + aneurysm_roundness/3/2, aneurysm_height*2/3 + aneurysm_roundness*2/3/2, 0],
          [-aneurysm_width*1/3 + aneurysm_roundness/3/5, aneurysm_height + aneurysm_roundness/3, 0],
          [aneurysm_width*1/3 - aneurysm_roundness/3/5, aneurysm_height + aneurysm_roundness/3, 0],
         [aneurysm_width - aneurysm_roundness/3/2, aneurysm_height*2/3 + aneurysm_roundness*2/3/2, 0],
        [aneurysm_width/10 + aneurysm_roundness/3, aneurysm_strecth, 0],
       [aneurysm_width/12 + aneurysm_intake/2, 0, 0],
      [aneurysm_width/4 + aneurysm_intake, 0, 0]
  ]

  x_vals, y_vals_top = generate_curve(control_points)
  x_vals = np.array(x_vals) + aneurysm_center

  y_vals_main = height + np.sum([i * k * np.sin(2 * np.pi * f/i * (x_vals) / length) for i in range(1, oscilations+1)], axis=0)
  y_min = y_vals_main.min()
  y_vals_top = y_min + np.array(y_vals_top)

  top_boundary_points = [fe.Point(x, y) for x, y in zip(x_vals[::-1], y_vals_top[::-1])]
  bottom_boundary_points = [fe.Point(0.0, height/2), fe.Point(length, height/2)]
  vessel_boundary_points = top_boundary_points + bottom_boundary_points

  aneurysm_domain = mshr.Polygon(vessel_boundary_points)
  vessel_domain = main_domain + aneurysm_domain

  return vessel_domain

In [43]:
def generate_vessel_geometry(vessel_type, length, height, params):

  match vessel_type:
    case 'healthy':
      undulation_amplitude = params['healthy']['undulation_amplitude']
      undulation_frequency = params['healthy']['undulation_frequency']
      oscilations = params['healthy']['oscilations']
      vessel_domain = create_healthy_domain(length, height, undulation_amplitude, undulation_frequency, oscilations)

    case 'isthmus':
      isthmus_height = params['isthmus']['isthmus_height']
      isthmus_width = params['isthmus']['isthmus_width']
      isthmus_position = params['isthmus']['isthmus_position']
      undulation_amplitude = params['isthmus']['undulation_amplitude']
      undulation_frequency = params['isthmus']['undulation_frequency']
      vessel_domain = create_isthmus_domain(length, height, isthmus_height, isthmus_width, isthmus_position, undulation_amplitude, undulation_frequency)

    case 'branch':
      branch_width = params['branch']['branch_width']
      branch_angle = params['branch']['branch_angle']
      branch_position_x = params['branch']['branch_position_x']
      undulation_amplitude = params['branch']['undulation_amplitude']
      undulation_frequency = params['branch']['undulation_frequency']
      oscilations = params['branch']['oscilations']
      vessel_domain = create_branch_domain(length, height, branch_width, branch_angle, branch_position_x, undulation_amplitude, undulation_frequency, oscilations)

    case 'aneurysm':
      aneurysm_width = params['aneurysm']['aneurysm_width']
      aneurysm_height = params['aneurysm']['aneurysm_height']
      aneurysm_strecth = params['aneurysm']['aneurysm_strecth']
      aneurysm_intake = params['aneurysm']['aneurysm_intake']
      aneurysm_roundness = params['aneurysm']['aneurysm_roundness']
      aneurysm_center = params['aneurysm']['aneurysm_center']
      undulation_amplitude = params['aneurysm']['undulation_amplitude']
      undulation_frequency = params['aneurysm']['undulation_frequency']
      oscilations = params['aneurysm']['oscilations']
      vessel_domain = create_aneurysm_domain(length, height, aneurysm_width, aneurysm_height, aneurysm_strecth, aneurysm_intake, aneurysm_roundness, aneurysm_center, undulation_amplitude, undulation_frequency, oscilations)

  return vessel_domain